In [1]:
%pip install pandas
%pip install requests
%pip install alpaca-py
%pip install pytz
%pip install pytickersymbols
%pip install pyarrow


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.3 -> 26.0.1
[noti

In [ ]:
from alpaca.data.timeframe import TimeFrame
from alpaca.data.requests import StockBarsRequest
from alpaca.data import StockHistoricalDataClient
from datetime import datetime
import pandas as pd
import os
import pyarrow

stock_client = StockHistoricalDataClient("PKDBAP3S6DPDWEUTLD77R256C6", "DVKUFxa3UNtu4yJuWXFWyuzGAEaqCkSgZvJvgBsectr1")


from alpaca.broker import client
from alpaca.trading.requests import GetAssetsRequest

#Note: I learned only AFTER writing this, you need seperate api keys for the broker client
# Get those here: https://broker-app.alpaca.markets/dashboard

broker_client = client.BrokerClient("CKZXMIOPRAG37W7T53KW5F2YOW", "8hA9KKQhF7ZWnFdb78Pb2dqenm6hBC1NUDqdUnHvzTH")

request_params = GetAssetsRequest(
    status="active",
    asset_class="us_equity",
    exchange="NYSE"
)

nyse_assets = broker_client.get_all_assets(request_params)
nasdaq_assets = broker_client.get_all_assets(GetAssetsRequest(status="active", asset_class="us_equity", exchange="NASDAQ"))

print(f"Active NYSE assets: {len(nyse_assets)}")
print(f"Active NASDAQ assets: {len(nasdaq_assets)}")

def fetch_symbols_from_response(nyse_assets, nasdaq_assets):
    #This is called list comprehension
    nyse_symbols = [asset.symbol for asset in nyse_assets if asset.status == "active" and asset.tradable]
    nasdaq_symbols = [asset.symbol for asset in nasdaq_assets if asset.status == "active" and asset.tradable]
    return nyse_symbols + nasdaq_symbols
symbols_list = fetch_symbols_from_response(nyse_assets, nasdaq_assets)
print(len(symbols_list), "Count of symbols fetched from broker API")

# ************************************* HAVE TO TALK WITH MADISON ABOUT THIS *********************************
# This is what I think we have to mess around with to get our top 1000 tickers 
# ************************************************************************************************************

def find_top_1000_assets(symbols_list, stock_client):
    # This function will fetch the latest daily dollar volume for each symbol and return the top 1000 by that metric.
    bars = stock_client.get_stock_bars(StockBarsRequest(
        symbol_or_symbols=symbols_list,
        timeframe=TimeFrame.Day,
        start=datetime(2026, 1, 1),
        end=datetime(2026, 2, 1)
    ))
    bars_df = bars.df
    bars_df["dollar_volume"] = bars_df["close"] * bars_df["volume"]
    # Sort symbols by dollar volume and take top 1000
    top_1000 = bars_df.groupby("symbol")["dollar_volume"].sum().nlargest(1000)
    return top_1000

top_1000_symbols = find_top_1000_assets(symbols_list, stock_client)
#print("Top 1000 symbols by latest daily dollar volume:")
#print(top_1000_symbols)

def get_stock_data(ticker):

    def get_historical_data(ticker):
        formatted_request = StockBarsRequest(
            symbol_or_symbols=[ticker], 
            start=datetime(2025, 4, 18), # This could change right now it is a hardcoded value but in the future it could be dynamic 
            end=datetime(2026,2,12), # This could change right now it is a hardcoded value but in the future it could be dynamic 
            timeframe=TimeFrame.Minute
        )
        response = stock_client.get_stock_bars(formatted_request)
        #print(response)
        return response


    mydf = get_historical_data(ticker).df

    #print(mydf.columns) # This is just to check what columns we have in the DataFrame it also is basically a test to make sure the above works
    #print(mydf.describe())

    mydf = mydf.reset_index()
    mydf = mydf.set_index("timestamp")

    mydf = mydf.tz_convert("US/Central")
    mydf = mydf.between_time("08:30", "15:00")
    mydf = mydf[mydf.index.dayofweek < 5]

    market_open = pd.to_datetime("08:30").time()
    market_close = pd.to_datetime("15:00").time()


    full_timeline = pd.date_range(start=mydf.index.min(),end=mydf.index.max(), freq="1min", tz="US/Central")
    full_timeline = full_timeline[(full_timeline.time >= market_open) & (full_timeline.time <= market_close) & (full_timeline.dayofweek < 5)]

    mydf = mydf.reindex(full_timeline)

    mydf[['open', 'high', 'low', 'close']] = mydf[['open', 'high', 'low', 'close']].ffill()
    mydf['volume'] = mydf['volume'].fillna(0)

    mydf = mydf.drop(columns=['trade_count','vwap'])

    mydf = mydf.reset_index()

    mydf = mydf.rename(columns={
        "index": "date",    
        "symbol": "ticker"
    })
    
    #print(mydf.describe())
    return mydf


def get_1000_stock_info():
    ticker_list = top_1000_symbols.index.tolist()[:5] # This [:5] at the end limits it to only the first 5 ticker symbols. When we want all thousand we just remove the [:5]

    all_dfs = []

    for ticker in ticker_list:
        df = get_stock_data(ticker)
        all_dfs.append(df)
        print(f"Finished {ticker}")
    
    final_df = pd.concat(all_dfs,ignore_index=True)

    return final_df


# This is temporary as its to test if my function works or not
tempdf = get_1000_stock_info()
print(tempdf)



# Below is all code for a parquet file which I think at the end we are supposed to have so I'm not deleting the code I have right now though at this stage it could be deleted.


# This is to create a parquet file (Sasha said she want's to take in a DataFrame)
"""
def create_parquet_files(symbols_list):
    os.makedirs("data/raw", exist_ok=True)
    
    for ticker in symbols_list:
        df = get_stock_data(ticker)
        df.to_parquet(f"data/raw/{ticker}.parquet", engine='pyarrow', compression='snappy')
"""

# This is commented out for now because it takes long to run as it builds the parquet files.
#create_parquet_files(top_1000_symbols.index.tolist())


# This is just to check that we can read the parquet files we just created and that they look correct. We will delete this code later, it is just for testing purposes.

"""parquet_files = os.listdir("data/raw")

for file in parquet_files:
    df = pd.read_parquet(f"data/raw/{file}", engine="pyarrow")
    ticker = file.replace(".parquet", "")
    print(f"{ticker} has {len(df)} rows of data")
    print(df)
    print(df.info())   
"""
# Sasha wants dataframe with ticker, date, open, high, low, close, volume

Active NYSE assets: 2883
Active NASDAQ assets: 5333
8214 Count of symbols fetched from broker API
Finished QQQ
Finished NVDA
Finished TSLA
Finished MSFT
Finished MU
                            date ticker     open    high       low    close  \
0      2025-04-21 08:30:00-05:00    QQQ  438.230  438.74  438.0800  438.685   
1      2025-04-21 08:31:00-05:00    QQQ  438.690  438.92  438.2200  438.560   
2      2025-04-21 08:32:00-05:00    QQQ  438.690  439.08  438.2800  438.720   
3      2025-04-21 08:33:00-05:00    QQQ  438.700  438.90  438.3700  438.550   
4      2025-04-21 08:34:00-05:00    QQQ  438.590  438.78  438.3500  438.730   
...                          ...    ...      ...     ...       ...      ...   
416410 2026-02-11 14:56:00-06:00     MU  409.980  410.16  409.2901  409.560   
416411 2026-02-11 14:57:00-06:00     MU  409.560  410.48  409.4900  410.400   
416412 2026-02-11 14:58:00-06:00     MU  410.355  411.00  410.2900  410.710   
416413 2026-02-11 14:59:00-06:00     MU  410.

'parquet_files = os.listdir("data/raw")\n\nfor file in parquet_files:\n    df = pd.read_parquet(f"data/raw/{file}", engine="pyarrow")\n    ticker = file.replace(".parquet", "")\n    print(f"{ticker} has {len(df)} rows of data")\n    print(df)\n    print(df.info())   \n'